In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os

# Set up imports
project_root = os.path.abspath("..") 
sys.path.append(project_root)  

# Enable efficient use of GPU memory
from config.gpu.gpu_utils import configure_tensorflow_gpu
configure_tensorflow_gpu()

In [ ]:
# 1. LOAD MODEL ARGUMENTS
NUM_FEATURES = 16
MAX_EPOCHS = 60
ES_PATIENCE = 15
DAYS_IN_SERIES = 120
DAYS_PER_BUCKET = 5
MAX_IMAGES_PER_SERIES = (DAYS_IN_SERIES // DAYS_PER_BUCKET) + 1
label_legend = ['Uncultivated', 'Cultivated', 'No Crop Growing', 'Soybeans', 'Rice', 'Corn', 'Cotton']

In [ ]:
# 2. LOAD TRAIN/VAL DATASETS
import tensorflow as tf
import datetime

if NUM_FEATURES == 16:
    train_ds = tf.data.Dataset.load("../data/train_ds_with_idx_16f")
    val_ds = tf.data.Dataset.load("../data/val_ds_with_idx_16f")
else:
    train_ds = tf.data.Dataset.load("../data/train_ds_with_idx_12f")
    val_ds = tf.data.Dataset.load("../data/val_ds_with_idx_12f")

In [ ]:
# 3.  ASSESS DATA QUALITY

from models.utils import count_labels

train_label_counts = count_labels(train_ds)
val_label_counts = count_labels(val_ds)

print("Train Label Distribution:")
for idx, label in enumerate(label_legend):
    print(f"{label}: {train_label_counts.get(idx, 0)}")

print("\nValidation Label Distribution:")
for idx, label in enumerate(label_legend):
    print(f"{label}: {val_label_counts.get(idx, 0)}")


In [ ]:
# Evaluate whether there is an imbalance in the distribution of labels by class
from models.utils import evaluate_class_balance
evaluate_class_balance(val_label_counts, label_legend, threshold=0.1)

In [ ]:
# 4. ARCHITECTURES ASSESSMENT (deeper or compact networks)
%load_ext tensorboard


### 1. Convolutional Neural Networks (CNN)

In [ ]:
from models.cnn import baseline_simplecnn

model_save_dir = f'../results/models/simplecnn_{DAYS_IN_SERIES}days.keras'
log_dir = "../results/logs/fit_simplecnn/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

model_simple, history_simple = baseline_simplecnn(
    train_ds=train_ds,
    val_ds=val_ds,
    label_legend=label_legend,
    max_images_per_series=MAX_IMAGES_PER_SERIES,
    num_features=NUM_FEATURES,
    xaxis_callback='epoch', # or 1 by batch
    max_epochs=MAX_EPOCHS,
    es_patience=ES_PATIENCE,
    model_save_dir=model_save_dir,
    model_name="baseline_simplecnn",
    log_dir=log_dir
)


In [ ]:
%tensorboard --logdir=../results/logs/fit_simplecnn

In [ ]:
from models.cnn import baseline_bigcnn

model_save_dir = f'../results/models/bigcnn_{DAYS_IN_SERIES}days.keras'
log_dir = "../results/logs/fit_bigcnn/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

model_simple, history_simple = baseline_bigcnn(
    train_ds=train_ds,
    val_ds=val_ds,
    label_legend=label_legend,
    max_images_per_series=MAX_IMAGES_PER_SERIES,
    num_features=NUM_FEATURES,
    xaxis_callback='epoch', # or 1 by batch
    max_epochs=MAX_EPOCHS,
    es_patience=ES_PATIENCE,
    model_save_dir=model_save_dir,
    model_name="baseline_bigcnn",
    log_dir=log_dir
)

In [ ]:
%tensorboard --logdir=../results/logs/fit_bigcnn/

### 2. Visual Geometry Group (VGG)

In [ ]:
from models.vgg import baseline_vgg1d

model_save_dir = f'../results/models/vgg1d_{DAYS_IN_SERIES}days.keras'
log_dir = "../results/logs/fit_vgg1d/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

model_vgg, history_vgg = baseline_vgg1d(
    train_ds=train_ds,
    val_ds=val_ds,
    label_legend=label_legend,
    max_images_per_series=MAX_IMAGES_PER_SERIES,
    num_features=NUM_FEATURES,
    xaxis_callback='epoch',
    max_epochs=MAX_EPOCHS,
    es_patience=ES_PATIENCE,
    model_save_dir=model_save_dir,
    model_name="baseline_vgg1d",
    log_dir=log_dir
)

In [ ]:
%tensorboard --logdir=../results/logs/fit_vgg1d/

In [ ]:
from models.vgg import baseline_vgg1d_compact

model_save_dir = f'../results/models/vgg1dcompact_{DAYS_IN_SERIES}days.keras'
log_dir = "../results/logs/fit_vgg1dcompact/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

model_vggcompact, history_vggcompact = baseline_vgg1d_compact(
    train_ds=train_ds,
    val_ds=val_ds,
    label_legend=label_legend,
    max_images_per_series=MAX_IMAGES_PER_SERIES,
    num_features=NUM_FEATURES,
    xaxis_callback='epoch',
    max_epochs=MAX_EPOCHS,
    es_patience=ES_PATIENCE,
    model_save_dir=model_save_dir,
    model_name="baseline_vgg1d_compact",
    log_dir=log_dir
)

In [ ]:
%tensorboard --logdir=../results/logs/fit_vgg1dcompact/

### 3. U-Net

In [ ]:
from models.unet import baseline_unet1d_light

model_save_dir = f'../results/models/unet1d_light_{DAYS_IN_SERIES}days.keras'
log_dir = "../results/logs/fit_unet1d_light/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

model_unet1d_light, history_unet1d_light = baseline_unet1d_light(
    train_ds=train_ds,
    val_ds=val_ds,
    label_legend=label_legend,
    max_images_per_series=MAX_IMAGES_PER_SERIES,
    num_features=NUM_FEATURES,
    xaxis_callback='epoch',
    max_epochs=MAX_EPOCHS,
    es_patience=ES_PATIENCE,
    model_save_dir=model_save_dir,
    model_name="baseline_unet1d_light",
    log_dir=log_dir
)

In [ ]:
%tensorboard --logdir=../results/logs/fit_unet1d_light/

In [ ]:
from models.unet import baseline_unet1d  

model_save_dir = f'../results/models/unet1d_{DAYS_IN_SERIES}days.keras'
log_dir = "../results/logs/fit_unet1d/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

model_unet1d, history_unet1d = baseline_unet1d(
    train_ds=train_ds,
    val_ds=val_ds,
    label_legend=label_legend,
    max_images_per_series=MAX_IMAGES_PER_SERIES,
    num_features=NUM_FEATURES,
    xaxis_callback='epoch',
    max_epochs=MAX_EPOCHS,
    es_patience=ES_PATIENCE,
    model_save_dir=model_save_dir,
    model_name="baseline_unet1d",
    log_dir=log_dir
)

In [ ]:
%tensorboard --logdir=../results/logs/fit_unet1d/

### 4. Residual Networks

In [ ]:
from models.resnet import baseline_resnet1d

model_save_dir = f'../results/models/resnet1d_{DAYS_IN_SERIES}days.keras'
log_dir = "../results/logs/fit_resnet1d/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

model_resnet, history_resnet = baseline_resnet1d(
    train_ds=train_ds,
    val_ds=val_ds,
    label_legend=label_legend,
    max_images_per_series=MAX_IMAGES_PER_SERIES,
    num_features=NUM_FEATURES,
    xaxis_callback='epoch',
    max_epochs=MAX_EPOCHS,
    es_patience=ES_PATIENCE,
    model_save_dir=model_save_dir,
    model_name="baseline_resnet1d",
    log_dir=log_dir
)

In [ ]:
%tensorboard --logdir=../results/logs/fit_resnet1d/

In [ ]:
from models.resnet import baseline_resnet1d

model_save_dir = f'../results/models/resnet1d_{DAYS_IN_SERIES}days.keras'
log_dir = "../results/logs/fit_resnet1d/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

model_resunet, history_resunet = baseline_resunet1d(
    train_ds=train_ds,
    val_ds=val_ds,
    label_legend=label_legend,
    max_images_per_series=MAX_IMAGES_PER_SERIES,
    num_features=NUM_FEATURES,
    xaxis_callback='epoch',
    use_time_channel=True, # add it to NUM_FEATURES (+ 1)
    max_epochs=MAX_EPOCHS,
    es_patience=ES_PATIENCE,
    model_save_dir=model_save_dir,
    model_name="baseline_resunet1d",
    log_dir=log_dir
)


In [ ]:
%tensorboard --logdir=../results/logs/fit_resunet1d/

### 5. Temporal Convolutional Network (TCN)

In [ ]:
from models.tcn import baseline_tcn

model_save_dir = f'../results/models/tcn_{DAYS_IN_SERIES}days.keras'
log_dir = "../results/logs/fit_tcn/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

model_tcn, history_tcn = baseline_tcn(
    train_ds=train_ds,
    val_ds=val_ds,
    label_legend=label_legend,
    max_images_per_series=MAX_IMAGES_PER_SERIES,
    num_features=NUM_FEATURES,
    xaxis_callback='epoch',
    max_epochs=MAX_EPOCHS,
    es_patience=ES_PATIENCE,
    model_save_dir=model_save_dir,
    model_name="baseline_tcn",
    log_dir=log_dir
)

In [ ]:
%tensorboard --logdir=../results/logs/fit_tcn/

### 6. Trasnformers

In [ ]:
### TRANSFORMER
from models.transformer import baseline_transformer1d

model_save_dir = f'../results/models/transformer1d_{DAYS_IN_SERIES}days.keras'
log_dir = "../results/logs/fit_transformer1d/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

model_transformer, history_transformer = baseline_transformer1d(
    train_ds=train_ds,
    val_ds=val_ds,
    label_legend=label_legend,
    max_images_per_series=MAX_IMAGES_PER_SERIES,
    num_features=NUM_FEATURES,
    xaxis_callback='epoch',
    max_epochs=MAX_EPOCHS,
    es_patience=ES_PATIENCE,
    model_save_dir=model_save_dir,
    model_name="baseline_transformer1d",
    log_dir=log_dir
)

In [ ]:
%tensorboard --logdir=../results/logs/fit_transformer1d/

In [ ]:
### CNN + TRANSFORMER
from models.transformer import baseline_cnn_transformer1d

model_save_dir = f'../results/models/cnn_transformer1d_{DAYS_IN_SERIES}days.keras'
log_dir = "../results/logs/fit_cnn_transformer1d/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

model_cnn_transformer, history_cnn_transformer = baseline_cnn_transformer1d(
    train_ds=train_ds,
    val_ds=val_ds,
    label_legend=label_legend,
    max_images_per_series=MAX_IMAGES_PER_SERIES,
    num_features=NUM_FEATURES,
    xaxis_callback='epoch',
    max_epochs=MAX_EPOCHS,
    es_patience=ES_PATIENCE,
    model_save_dir=model_save_dir,
    model_name="baseline_cnn_transformer1d",
    log_dir=log_dir
)

In [ ]:
%tensorboard --logdir=../results/logs/fit_cnn_transformer1d/

### 7. Efficient Net

In [ ]:
from models.efficientnet import baseline_efficientnet1d

model_save_dir = f'../results/models/efficientnet1d_{DAYS_IN_SERIES}days.keras'
log_dir = "../results/logs/fit_efficientnet1d/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

model_efficientnet, history_efficientnet = baseline_efficientnet1d(
    train_ds=train_ds,
    val_ds=val_ds,
    label_legend=label_legend,
    max_images_per_series=MAX_IMAGES_PER_SERIES,
    num_features=NUM_FEATURES,
    xaxis_callback='epoch',
    max_epochs=MAX_EPOCHS,
    es_patience=ES_PATIENCE,
    model_save_dir=model_save_dir,
    model_name="baseline_efficientnet1d",
    log_dir=log_dir
)

In [ ]:
%tensorboard --logdir=../results/logs/fit_efficientnet1d/

### 8. Inception

In [ ]:
from models.inception import baseline_inception1d

model_save_dir = f'../results/models/inception1d_{DAYS_IN_SERIES}days.keras'
log_dir = "../results/logs/fit_inception1d/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

model_inception, history_inception = baseline_inception1d(
    train_ds=train_ds,
    val_ds=val_ds,
    label_legend=label_legend,
    max_images_per_series=MAX_IMAGES_PER_SERIES,
    num_features=NUM_FEATURES,
    xaxis_callback='epoch',
    max_epochs=MAX_EPOCHS,
    es_patience=ES_PATIENCE,
    model_save_dir=model_save_dir,
    model_name="baseline_inception1d",
    log_dir=log_dir
)

In [ ]:
%tensorboard --logdir=../results/logs/fit_inception1d/

In [ ]:
from models.inception import baseline_inception1d_se_augmented

model_save_dir = f'../results/models/inception1d_se_augmented_{DAYS_IN_SERIES}days.keras'
log_dir = "../results/logs/fit_inception1d_se_augmented/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

model_inception, history_inception = baseline_inception1d_se_augmented(
    train_ds=train_ds,
    val_ds=val_ds,
    label_legend=label_legend,
    max_images_per_series=MAX_IMAGES_PER_SERIES,
    num_features=NUM_FEATURES,
    xaxis_callback='epoch',
    max_epochs=MAX_EPOCHS,
    es_patience=ES_PATIENCE,
    model_save_dir=model_save_dir,
    model_name="baseline_inception1d_se_augmented",
    log_dir=log_dir,
    augment=True
)

In [ ]:
%tensorboard --logdir=../results/logs/fit_inception1d_se_augmented/

In [ ]:
from models.inception import baseline_inception1d_se_mixup_focal_attention_residual

model_save_dir = f'../results/models/inception1d_se_mixup_focal_attention_residual_{DAYS_IN_SERIES}days.keras'
log_dir = "../results/logs/fit_inception1d_se_mixup_focal_attention_residual/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

model_inception, history_inception = baseline_inception1d_se_mixup_focal_attention_residual(
    train_ds=train_ds,
    val_ds=val_ds,
    label_legend=label_legend,
    max_images_per_series=MAX_IMAGES_PER_SERIES,
    num_features=NUM_FEATURES,
    xaxis_callback='epoch',
    max_epochs=MAX_EPOCHS,
    es_patience=ES_PATIENCE,
    model_save_dir=model_save_dir,
    model_name="baseline_inception1d_se_mixup_focal_attention_residual",
    log_dir=log_dir,
    apply_oversampling=False,
    apply_mixup=True
)

In [ ]:
%tensorboard --logdir=../results/logs/fit_inception1d_se_mixup_focal_attention_residual/